# VA Group Project / 2020W. 
## Group N+1

### Dataset Overview 

* Our goal is to explore the trends and correlations between different datasets, available on [Gap Minder](https://www.gapminder.org/).
* Every dataset contains differnt subsets of countries / years.
* The original data is in the form: one row per country with values for every year as columns. The years' columns are transformed into rows to make the data processing more convenient.
* All datasets of interest are merged to make the data exploration easier.

In [ ]:
#disable some annoying warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#plots the figures in place instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np

import altair as alt

import ipywidgets as widgets

# Load Data

## Helper functions for data loading 

In [ ]:

def getYearsOfInterest(fromYear, toYear):
    """
    Generates a range of years [fromYear, toYear]
    """
    return [str(x) for x in range(fromYear, toYear+1)]

def filterData(valueColumns, metaDataColumns, data):
    """
    Filter valueColumns + metaDataColumns from the data frame
    All missing valueColumns are added to the resulting data frame (with value = None)
    """
    missingColumns = list(set(valueColumns) - set(data.columns))
    for c in missingColumns:
        data[c] = None 
    return data[list(set(metaDataColumns) | set(valueColumns))]

def unpivot(data, key_columns, data_column, value_column):
    """
    Transforms all non key_columns into rows
    """
    return pd.melt(data, id_vars=key_columns, var_name=data_column, value_name=value_column)

def loadSingleDataset(path, from_year, to_year, key_columns, data_column, value_column):
    """
    Loads a single data set from csv
    """
    data = pd.read_csv(path) 
    data = filterData(getYearsOfInterest(from_year, to_year), key_columns, data)
    return unpivot(data, key_columns, data_column, value_column)

def mergeDatasets(datasets, keys):
    """
    Merge datasets using keys as key columns
    The merge operation is outer join of all data sets
    """
    data = datasets[0]
    
    for i in range(1, len(datasets)):
        data = data.merge(datasets[i], how='outer', left_on=keys, right_on=keys)
        
    return data

In [ ]:
# global report params 
FROM_YEAR = 1900
TO_YEAR   = 2020

In [ ]:
gdp_growth = loadSingleDataset('data/gdp_total_yearly_growth.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'gdp_growth')
gdp_growth.head()


In [ ]:
children_per_woman_total_fertility = loadSingleDataset('data/children_per_woman_total_fertility.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'children_per_woman_total_fertility')
children_per_woman_total_fertility.head()

In [ ]:
co2_emissions_tonnes_per_person = loadSingleDataset('data/co2_emissions_tonnes_per_person.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'co2_emissions_tonnes_per_person')
co2_emissions_tonnes_per_person.head()


In [ ]:
mean_years_in_school_women_percent_men_25_to_34_years = loadSingleDataset('data/mean_years_in_school_women_percent_men_25_to_34_years.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'mean_years_in_school_women_percent_men_25_to_34_years')
mean_years_in_school_women_percent_men_25_to_34_years.head()

In [ ]:
average_age_of_dollar_billionaires_years = loadSingleDataset('data/average_age_of_dollar_billionaires_years.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'average_age_of_dollar_billionaires_years')
average_age_of_dollar_billionaires_years.head()

In [ ]:
food_supply= loadSingleDataset('data/food_supply.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'food_supply')
food_supply.head()

In [ ]:
hourly_compensation = loadSingleDataset('data/hourly_compensation.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'hourly_compensation')
hourly_compensation.head()

In [ ]:
income_per_person= loadSingleDataset('data/income_per_person.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'income_per_person')
income_per_person.head()

In [ ]:
suicide_per_100000_people = loadSingleDataset('data/suicide_per_100000_people.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'suicide_per_100000_people')
suicide_per_100000_people.head()

In [ ]:
total_number_of_dollar_billionaires = loadSingleDataset('data/total_number_of_dollar_billionaires.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'total_number_of_dollar_billionaires')
total_number_of_dollar_billionaires.head()

In [ ]:
working_hours_per_week = loadSingleDataset('data/working_hours_per_week.csv', 
                               FROM_YEAR, TO_YEAR, 
                               ['country'], 
                               'year', 
                               'working_hours_per_week')
working_hours_per_week.head()

## The final merged dataset

* Call mergeDatasets function to form the final dataset
* Augment data with additional attributes (i.e. continent and region data for _'country'_ and decade for _'year'_)

In [ ]:
# merge the datasets in one that contains all the data
data = mergeDatasets([
    gdp_growth, 
    children_per_woman_total_fertility,
    co2_emissions_tonnes_per_person,
    mean_years_in_school_women_percent_men_25_to_34_years,
    average_age_of_dollar_billionaires_years,
    food_supply,
    hourly_compensation,
    income_per_person,
    suicide_per_100000_people,
    total_number_of_dollar_billionaires,
    working_hours_per_week
], ['country', 'year'])

data.sort_values(by=['country', 'year'], inplace=True, ignore_index=True)


countries = pd.read_csv('data/countryContinent.csv')

data = data.merge(countries, how='left', left_on=['country'], right_on=['country'])
data = data.convert_dtypes()

#add 'decade' computed column 
data['decade'] = data['year'].str.slice(0, 3)  + '0'

#check for missing countries (they have to be corrected in countryContinent.csv)
missing_countries = data[data["region_code"].isnull()]['country'].unique()

if (len(missing_countries) == 0):
    print("Country mapping is OK")
else:
    print(missing_countries)
    
data.to_csv('data/data.csv')

In [ ]:
# basic statistics of the loaded data 
print(data.count())
data.head(50)

## Helper variables for different set of columns in the dataset

In [ ]:
# change all _ with line breaks - in this case it's easier to display column names in the plots
mapping = {}
for col in data:
    mapping[col] = col.replace('_', "\n")
    
data = data.rename(columns=mapping)

key_columns = ['country', 'year']

measure_columns = [
        "gdp\ngrowth",
        "children\nper\nwoman\ntotal\nfertility",
        "co2\nemissions\ntonnes\nper\nperson",
        "mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears",
        "average\nage\nof\ndollar\nbillionaires\nyears",
        "food\nsupply",
        "hourly\ncompensation",
        "income\nper\nperson",
        "suicide\nper\n100000\npeople",
        "total\nnumber\nof\ndollar\nbillionaires",
        "working\nhours\nper\nweek"
    ]

all_columns = key_columns + measure_columns

# Show Data

## Data Completeness
 > In the data quality framework, data completeness refers to the degree to which all data in a data set is available. A measure of data completeness is the percentage of missing data entries [[1]](https://dataladder.com/missing-data-and-data-completeness/)

In [ ]:

t1 = pd.melt(data[all_columns], id_vars=['country', 'year'], var_name=['measure'], value_name='val')
t1['Countries Count'] = t1['val'].isnull() 

t1 = t1.groupby(['year', 'measure'])['Countries Count'].sum().reset_index()

alt.Chart(t1).mark_rect().encode(
    x='year:O',
    y='measure:O',
    color='Countries Count:Q'
).properties(
    width=800,
    height=300,
    title='Data Completeness'
)


## Selection of Attributes.

The aim is to provide a short description and overall visualization of the attributes/columns that has been chosen and is used within this template.

* gdp_growth (gdp_growth): yearly growth of GDP
* children_per_woman_total_fertility (fertility): number of children per woman
* co2_emissions_tonnes_per_person (co2_emissions): carbon dioxide emission from burning of fossil fueles in tonnes per person
* mean_years_in_school_women_percent_men_25_to_34_years (school_years): the average number of years attended in school for women and men age 25 to 34 (including primary, secondary, tertiary education)
* average_age_of_dollar_billionaires_years (age_billionaires): average age of dollar billionaires in the country of their citizenship
* food_supply (calories): kilocalories intake per person per day (normally 1.500-3.000 kcal/day)
* hourly_compensation (compensation): average hourly labor cost per employee
* income_per_person (income): GDP per person PPP and inflation adjusted
* suicide_per_100000_people (suicide): mortality due to self-inflicted injury per 100.000 people 
* total_number_of_dollar_billionaires (billionaires): total number of dollar billionaires in the country of their citizenship
* working_hours_per_week (working_hours): total amount of yearly working hours divided by 52 weeks


In [ ]:
# code her for Johannes' interactive scatter plot matrix

### Interpretation

short paragraph on why, how and what of interactive scatterplot matrix.

# DESCRIPTIVE STATISTICS.

Analyzing our dataset using descriptive statistics on the level of individual attributes.
This includes simple plots of distributions and statistics.


# Overview Data.




In [ ]:
# Hendrik's code here for world map
import geopandas as gpd

data['year'] = data['year'].astype(int)
gdf = gpd.read_file('data/CNTR_RG_60M_2020_4326.shp')
gdf3=gdf[gdf.NAME_ENGL!='Antarctica']
gdf3
toggle = widgets.RadioButtons(options=['alphabetically','descending','ascending'], description="Bar sorting")
print('Please choose desired sorting, data, year and region')
print('For comparing individual countries: shift-button + mouse-click on desired countries')
@widgets.interact(Continent=['World','Europe','Asia','Americas','Africa','Oceania'],Year=(1900,2020),Attribute=['gdp\ngrowth','children\nper\nwoman\ntotal\nfertility','co2\nemissions\ntonnes\nper\nperson','mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears','average\nage\nof\ndollar\nbillionaires\nyears','food\nsupply','hourly\ncompensation','income\nper\nperson','suicide\nper\n100000\npeople','total\nnumber\nof\ndollar\nbillionaires','working\nhours\nper\nweek'],sorting=toggle)
def react(sorting,Attribute,Year,Continent):
        gdf4=gdf3
        ee =data.loc[data['year']==Year]
        dd = ee.rename(columns = {"code\n3": "ISO3_CODE"})
        multi = alt.selection_multi(fields=['ISO3_CODE'])
        color = alt.condition(multi,
                          alt.Color(Attribute+':Q', 
                          scale=alt.Scale(scheme='blues')),
                          alt.value('lightgray'))
        brush = alt.selection_interval()
        hover = alt.selection_single( on='mouseover',fields=['ISO3_CODE'])
        if sorting=='alphabetically':
            rank='country'
        if sorting=='descending':
            rank=alt.Y('country', sort='-x')
        if sorting=='ascending':
            rank=alt.Y('country', sort='x')       
        if Continent!='World':
            continent=dd.loc[dd['continent']==Continent]
            europa=continent.loc[:,'ISO3_CODE'].values
            mapContinent=gdf3[gdf3.ISO3_CODE.isin(europa)]
            gdf4=mapContinent
            dd=dd.loc[dd['ISO3_CODE'].isin(europa)]
            gg=dd
            region=dd['sub\nregion'].unique()
            l=list(region)
            l.insert(0,'Continent')
            
            @widgets.interact(Region=l)
            def back(Region):
                if Region!='Continent':
                    continent2=gg.loc[gg['sub\nregion']==Region]
                    europa=continent2.loc[:,'ISO3_CODE'].values
                    mapContinent=gdf3[gdf3.ISO3_CODE.isin(europa)]
                    gdf4=mapContinent
                    hh=gg.loc[dd['ISO3_CODE'].isin(europa)]
                    map = alt.Chart(gdf4).mark_geoshape(stroke='lightgray').encode(color=color,tooltip=['NAME_ENGL',Attribute+':Q']
                        ).transform_lookup(lookup='ISO3_CODE',from_=alt.LookupData(dd, 'ISO3_CODE', [Attribute])).add_selection(multi
                        ).properties(width=650,height=400
                        ).properties(title='Overview')
                    bars = alt.Chart(hh).mark_bar(size=10).encode( y=rank, x=Attribute, tooltip=[Attribute+':Q'],color=alt.Color(Attribute+':Q',scale=alt.Scale(scheme='blues'))
                                    ).add_selection(multi).transform_filter(multi).properties(title='Countries')
                    text = alt.Chart(hh).mark_text(size=10, align='left', baseline='middle', dx=3  
                            ).encode( y=rank, x=Attribute,text=Attribute+':Q'
                            ).transform_filter(multi)
                    return  map&(bars+text)
                if Region=='Continent':
                    continent=gg.loc[gg['continent']==Continent]
                    europa=continent.loc[:,'ISO3_CODE'].values
                    mapContinent=gdf3[gdf3.ISO3_CODE.isin(europa)]
                    gdf4=mapContinent
                    jj=gg.loc[dd['ISO3_CODE'].isin(europa)]
                    region=dd['sub\nregion'].unique()
                    l=list(region)
                    l.insert(0,'Continent')
                    map = alt.Chart(gdf4).mark_geoshape(stroke='white'
                    ).encode(color=color,tooltip=['NAME_ENGL',Attribute+':Q']
                    ).transform_lookup(lookup='ISO3_CODE',from_=alt.LookupData(dd, 'ISO3_CODE', [Attribute])
                    ).add_selection( multi
                        ).properties( width=650, height=400
                    ).properties(title='Overview')
                    bars = alt.Chart(jj).mark_bar(size=10).encode( y=rank, x=Attribute, tooltip=[Attribute+':Q'],color=alt.Color(Attribute+':Q',scale=alt.Scale(scheme='blues'))
                                    ).add_selection( multi
                                    ).transform_filter(multi
                                    ).properties(title='Countries')
                    text = alt.Chart(jj).mark_text(size=10, align='left', baseline='middle',dx=3  
                            ).encode( y=rank, x=Attribute,text=Attribute+':Q'
                            ).transform_filter(multi )
                    return  map&(bars+text)
        if Continent=='World':
            map = alt.Chart(gdf4).mark_geoshape(stroke='white'
                        ).encode(color=color, tooltip=['NAME_ENGL',Attribute+':Q']
                        ).transform_lookup(lookup='ISO3_CODE',from_=alt.LookupData(dd, 'ISO3_CODE', [Attribute])
                        ).add_selection( multi
                            ).properties( width=650,height=400
                        ).properties(title='Overview')
            bars = alt.Chart(dd).mark_bar(size=10).encode( y=rank, x=Attribute, tooltip=[Attribute+':Q'],color=alt.Color(Attribute+':Q',scale=alt.Scale(scheme='blues'))
                            ).add_selection( multi
                            ).transform_filter(multi
                        ).properties(title='Countries')
            text = alt.Chart(dd).mark_text(size=10,align='left',baseline='middle', dx=3  
                    ).encode(y=rank, x=Attribute,text=Attribute+':Q'
                    ).transform_filter(multi)
            return  map&(bars+text)
       
            

### Interpretation

The world map was created to give an overview of the data series. The strength of the colors expresses the characteristics of the selected attribute. The aim is to visualize the differences between different countries. The different data series can be selected using the drop-down menu. The desired year is set with the slider. Individual continents or regions can be selected using additional drop-down menus. The exact values of the data can be read from the bar plot below the world map. The user can select several countries on the map (shift + click) in order to compare them individually in the barplot.

In [ ]:
# Hendrik's code here for boxplot and density plot
import matplotlib.pyplot as plt
import seaborn as sns

print('Please choose desired data, year and granularity')
@widgets.interact(Year=(1900,2020),Attribute=['gdp\ngrowth','children\nper\nwoman\ntotal\nfertility','co2\nemissions\ntonnes\nper\nperson','mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears','average\nage\nof\ndollar\nbillionaires\nyears','hourly\ncompensation','suicide\nper\n100000\npeople','working\nhours\nper\nweek'], Resolution=(5, 25, 5))
def react(Attribute,Year,Resolution):
    indicator=Attribute
    slider=Year
    bins=Resolution
    con=data
    con2=con.loc[data['year']==slider]
    
    ind=indicator
    con2.dropna()
    plt.figure(figsize=(20,10))
    plt.subplot(121)
    
    sns.boxplot(x='continent',y=ind,data=con2) 
    plt.title("Boxplots per Continent", size=24)
    plt.subplot(122)
    if con2[ind].isnull().all():
        print('no data')
    else:
        sns.distplot(con2[ind],bins=bins)
        plt.title("Denityplot world", size=24)
    

### Interpretation


The box plots are intended to give an overview of the distribution of a specific data series. The continents are shown separately so that a comparison is possible. A desity plot is shown on the right to visualize the data series worldwide. The desired data series can be selected using a drop-down menu. The year can be set using a slider. The resolution of the density plot can also be adjusted using a slider.

### CORRELATIONS.

Analyzing ourdataset by looking at correlations between attributes (dimensions) and coming up with an interpretation why in which way specific attributes are correlated. 

In [ ]:
#Hendrik's code for scatterplot adjusted for cutting out left plot (development)
print('Please choose desired attributes and a country')
print('Mouse: panning and zooming')
@widgets.interact(Year=(1900,2020),Country=data['country'].unique(), Attribute1=['gdp\ngrowth','children\nper\nwoman\ntotal\nfertility','co2\nemissions\ntonnes\nper\nperson','mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears','average\nage\nof\ndollar\nbillionaires\nyears','food\nsupply','hourly\ncompensation','income\nper\nperson','suicide\nper\n100000\npeople','total\nnumber\nof\ndollar\nbillionaires','working\nhours\nper\nweek'], Attribute2=['co2\nemissions\ntonnes\nper\nperson','gdp\ngrowth','children\nper\nwoman\ntotal\nfertility','mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears','average\nage\nof\ndollar\nbillionaires\nyears','food\nsupply','hourly\ncompensation','income\nper\nperson','suicide\nper\n100000\npeople','total\nnumber\nof\ndollar\nbillionaires','working\nhours\nper\nweek'], Attribute3=['income\nper\nperson','gdp\ngrowth','children\nper\nwoman\ntotal\nfertility','co2\nemissions\ntonnes\nper\nperson','mean\nyears\nin\nschool\nwomen\npercent\nmen\n25\nto\n34\nyears','average\nage\nof\ndollar\nbillionaires\nyears','food\nsupply','hourly\ncompensation','suicide\nper\n100000\npeople','total\nnumber\nof\ndollar\nbillionaires','working\nhours\nper\nweek'])
def react(Country,Attribute1,Attribute2,Attribute3, Year):
    slider=Year
    country=Country
    con11=data.loc[data['country']==country]
    con22=con11.loc[data['year'].between(slider-10, slider+10)]
    con=data
    con2=con.loc[data['year']==slider]
    ind=Attribute1
    ind2=Attribute2
    ind3=Attribute3
    bubble= alt.Chart(con2).mark_circle().encode(
                x=alt.X(ind),
                y=ind2,
                color='continent',
                size=ind3,
                tooltip=['country',ind+':Q',ind2+':Q', ind3+':Q']
            ).properties(title='Attributes 1-3 \n(1:x-Axis 2:y-Axis 3: Bubble)').interactive() 
    text= ( alt.Chart(con11.loc[data['year']==slider])
        .mark_text(dy=-5)
        .encode(x=alt.X(ind), y=ind2, text=alt.Text("country:N")))          
    return (bubble+text)

### Interpretation

short paragraph on why, how and what of scatterplot.

Interestingly a lot of Asian countries that are not mentioned a lot in the media have high income per person in the early 60's and keep the status throughout the 2000's (Brunei, Kuwait, Saudi Arabia). With United Arab Emirates speeding up in the end of the 60's (with forming of UAE) and overaking them in less than a couple of years. As expexted calories intake as well as Co2 emission keep increasing with those countries.
African countries increase their income per person as well but not as much as their food supply.

Contrary to the hypothesis that income per person and working hours would have an impact on suicide rate the heatmap shows no or even a negative correlation. Meaning money is not the sole reason why people decide to end their lives.
Income per person on the other hand is highly correlated to calories intake and co2 emission per person which is comprehensible given the fact that with higher income there is more money to spend for consumation. 
Interestingly calories intake is also connected to the mean years in school which can be explained by the implicit relationship between calories intake and income per person. 
The average age of dollar billionaires is highly correlated to the hourly compensation which makes sense.

### CLUSTERING.

Clustering similar items and show the clustering results.
1. User can interactively select the cluster algorithms and/or its parameters.
2. User can select one/more cluster/s from the resulting visualization.
3. User can see the selected data with its cluster affiliation in a second interactive visualization (simple overview+detail visualization setup)



In [ ]:
# Johannes' code for T-SNE, PCA,...

### Interpretation

short paragraph on why, how and what of cluster algorithm.

